# Fine Tuning T5 for Customer Service

In [1]:
import os
import torch
import time
import numpy as np 
import pandas as pd
import tensorflow as tf
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration, GenerationConfig, TrainingArguments, Trainer

2024-10-07 05:09:11.215096: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Set configuration parameters

In [2]:
model_name='t5-small'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'  

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = T5ForConditionalGeneration.from_pretrained(model_name)
base_model = base_model.to(device)

Load datasets

In [4]:
dataset_bitext_train = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train[:24000]")
dataset_bitext_test = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train[-6000:-3000]")
dataset_bitext_validation = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train[-3000:]")

Using the latest cached version of the dataset since bitext/Bitext-customer-support-llm-chatbot-training-dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/johnmoses/.cache/huggingface/datasets/bitext___bitext-customer-support-llm-chatbot-training-dataset/default/0.0.0/430d1a89bd93bd1fa23c16f29dd53e73f0087443 (last modified on Thu Sep  5 17:13:46 2024).
Using the latest cached version of the dataset since bitext/Bitext-customer-support-llm-chatbot-training-dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/johnmoses/.cache/huggingface/datasets/bitext___bitext-customer-support-llm-chatbot-training-dataset/default/0.0.0/430d1a89bd93bd1fa23c16f29dd53e73f0087443 (last modified on Thu Sep  5 17:13:46 2024).
Using the latest cached version of the dataset since bitext/Bitext-customer-support-llm-chatbot-training-dataset couldn't be found on the Hugging Face Hub
Fou

Merge datasets

In [5]:
dataset_train_merged = concatenate_datasets(
    [dataset_bitext_train]
)
dataset_test_merged = concatenate_datasets(
    [dataset_bitext_test]
)
dataset_validation_merged = concatenate_datasets(
    [dataset_bitext_validation]
)

Save datasets locally as `.csv` files

In [6]:
dataset_train_merged.to_csv('train_merged.csv', index=False)
dataset_test_merged.to_csv('test_merged.csv', index=False)
dataset_validation_merged.to_csv('validation_merged.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

2111954

Let's look at each dataset

In [12]:
dataset_train_merged

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 24000
})

In [6]:
dataset_test_merged

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 2000
})

In [9]:
dataset_validation_merged

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 1000
})

Load merged dataset from disk

In [7]:
dataset = load_dataset('csv', data_files={
    "train": "train_merged.csv", 
    "test": "test_merged.csv", 
    "validation": "validation_merged.csv"
    })

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 3000
    })
})

In [9]:
dataset['train'][0]

{'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'category': 'ORDER',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}

Explore dataset

In [10]:
dataset_train = pd.read_csv('./train_merged.csv')

In [11]:
dataset_train.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token="<|im_end|>")

In [13]:
tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

Try to translate English to German

In [14]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = base_model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
<pad> Wie alt sind Sie?</s>


/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Get instruction prompt using an index

In [15]:
def get_prompt(data, index):
    instruction = data.iloc[index]['instruction']
    response = data.iloc[index]['response']
    
    prompt = f'''
    instruction:
    {instruction}
    
    response:
    {response}
    
    '''
    
    return prompt

def get_instruction(data,index):
    instruction = data.iloc[index]['instruction']
    return f'''
    
    instruction:
    {instruction}
    
    response:
    
    '''

Check model output with one shot

In [16]:
test_prompt = get_prompt(dataset_train,0) + get_instruction(dataset_train,10)
print(test_prompt)


    instruction:
    question about cancelling order {{Order Number}}
    
    response:
    I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.
    
    
    
    instruction:
    i dont know what to do to cancel order {{Order Number}}
    
    response:
    
    


In [17]:
input_text = tokenizer.encode(test_prompt,return_tensors="pt")
output = tokenizer.decode(base_model.generate(input_text)[0])
print(output)

<pad><extra_id_0>: I have understood you have a question regarding cancelling order <unk>Order Number


In [18]:
idx = 100
idx_var = 50

check = dataset_train.iloc[idx]['instruction']
check2 = dataset_train.iloc[idx]['response']

test = dataset_train.iloc[idx_var]['instruction']
test_response = dataset_train.iloc[idx_var]['response']

print(f'instruction:\n{test}\n\nresponse:\n')

print(f'======================\nexpected reponse: \n {test_response}')

instruction:
canceling order {{Order Number}}

response:

expected reponse: 
 I appreciate that you're seeking assistance with canceling your order tied to the order number {{Order Number}}, and I'm here to guide you through the cancellation process. Please follow these steps:

1. Access Your Account: Log in to your {{Online Company Portal Info}} using your username and password.
2. Navigate to Your Orders: Once you're logged in, locate the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate the Order: Look for the specific order associated with the order number {{Order Number}}.
4. Initiate the Cancellation: Within the order details, you should find an option to '{{Online Order Interaction}}'. Click on it to start the cancellation process.
5. Confirm the Cancellation: The system may ask for your confirmation or gather feedback. Please provide the necessary information accurately.

If you encounter any difficulties or have further questions, our dedicate

Tokenize function

In [19]:
def tokenize_function(example):
    start_prompt = 'Instruction:\n'
    end_prompt = '\nResponse:'
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["instruction"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["response"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

In [20]:
shuffled_dataset = dataset.shuffle(seed=42)

In [21]:
tokenized_datasets = shuffled_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['flags', 'instruction', 'category', 'intent', 'response'])

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3000
    })
})

Fine Tune

In [23]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
finetuned_model = finetuned_model.to('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [24]:
output_dir = 't5-customer-service-log'

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=5e-3,
    num_train_epochs=2,
    per_device_train_batch_size=16,     # batch size per device during training
    per_device_eval_batch_size=16,      # batch size for evaluation
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',        # evaluation strategy to adopt during training
    eval_steps=500,                 
)

trainer = Trainer(
    model=finetuned_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Evaluate the trainer

In [25]:
trainer.evaluate()

  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 3.600660800933838,
 'eval_model_preparation_time': 0.0019,
 'eval_runtime': 115.4695,
 'eval_samples_per_second': 25.981,
 'eval_steps_per_second': 1.628}

In [26]:
%%time

trainer.train()

  0%|          | 0/3000 [00:00<?, ?it/s]

{'loss': 1.1855, 'grad_norm': 0.16349054872989655, 'learning_rate': 0.004916666666666666, 'epoch': 0.03}
{'loss': 0.4826, 'grad_norm': 0.10131632536649704, 'learning_rate': 0.004833333333333334, 'epoch': 0.07}
{'loss': 0.4295, 'grad_norm': 0.12298038601875305, 'learning_rate': 0.00475, 'epoch': 0.1}
{'loss': 0.4072, 'grad_norm': 0.09317249804735184, 'learning_rate': 0.004666666666666667, 'epoch': 0.13}
{'loss': 0.3787, 'grad_norm': 0.12126368284225464, 'learning_rate': 0.004583333333333333, 'epoch': 0.17}
{'loss': 0.3543, 'grad_norm': 0.07475490868091583, 'learning_rate': 0.0045000000000000005, 'epoch': 0.2}
{'loss': 0.3445, 'grad_norm': 0.0923856794834137, 'learning_rate': 0.004416666666666667, 'epoch': 0.23}
{'loss': 0.3475, 'grad_norm': 0.09289425611495972, 'learning_rate': 0.004333333333333334, 'epoch': 0.27}
{'loss': 0.3121, 'grad_norm': 0.10281085222959518, 'learning_rate': 0.00425, 'epoch': 0.3}
{'loss': 0.3135, 'grad_norm': 0.10240963846445084, 'learning_rate': 0.00416666666666

  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 0.39064279198646545, 'eval_model_preparation_time': 0.0019, 'eval_runtime': 115.088, 'eval_samples_per_second': 26.067, 'eval_steps_per_second': 1.634, 'epoch': 0.33}
{'loss': 0.3198, 'grad_norm': 0.086480051279068, 'learning_rate': 0.004083333333333333, 'epoch': 0.37}
{'loss': 0.3155, 'grad_norm': 0.11693891882896423, 'learning_rate': 0.004, 'epoch': 0.4}
{'loss': 0.2991, 'grad_norm': 0.08312585949897766, 'learning_rate': 0.003916666666666666, 'epoch': 0.43}
{'loss': 0.2983, 'grad_norm': 0.08346576243638992, 'learning_rate': 0.0038333333333333336, 'epoch': 0.47}
{'loss': 0.2991, 'grad_norm': 0.08641285449266434, 'learning_rate': 0.00375, 'epoch': 0.5}
{'loss': 0.2799, 'grad_norm': 0.07280765473842621, 'learning_rate': 0.0036666666666666666, 'epoch': 0.53}
{'loss': 0.2873, 'grad_norm': 0.08434479683637619, 'learning_rate': 0.0035833333333333333, 'epoch': 0.57}
{'loss': 0.2637, 'grad_norm': 0.0968218743801117, 'learning_rate': 0.0034999999999999996, 'epoch': 0.6}
{'loss': 

  0%|          | 0/188 [00:00<?, ?it/s]

...

  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 0.3091459274291992, 'eval_model_preparation_time': 0.0019, 'eval_runtime': 115.4163, 'eval_samples_per_second': 25.993, 'eval_steps_per_second': 1.629, 'epoch': 1.67}
{'loss': 0.2207, 'grad_norm': 0.07797810435295105, 'learning_rate': 0.00075, 'epoch': 1.7}
{'loss': 0.2211, 'grad_norm': 0.08147252351045609, 'learning_rate': 0.0006666666666666666, 'epoch': 1.73}
{'loss': 0.2204, 'grad_norm': 0.07256118953227997, 'learning_rate': 0.0005833333333333334, 'epoch': 1.77}
{'loss': 0.2175, 'grad_norm': 0.0893479660153389, 'learning_rate': 0.0005, 'epoch': 1.8}
{'loss': 0.2285, 'grad_norm': 0.08022065460681915, 'learning_rate': 0.00041666666666666664, 'epoch': 1.83}
{'loss': 0.2184, 'grad_norm': 0.06792289763689041, 'learning_rate': 0.0003333333333333333, 'epoch': 1.87}
{'loss': 0.2176, 'grad_norm': 0.06807844340801239, 'learning_rate': 0.00025, 'epoch': 1.9}
{'loss': 0.2203, 'grad_norm': 0.07047216594219208, 'learning_rate': 0.00016666666666666666, 'epoch': 1.93}
{'loss': 0.2161,

  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 0.3029376268386841, 'eval_model_preparation_time': 0.0019, 'eval_runtime': 115.2077, 'eval_samples_per_second': 26.04, 'eval_steps_per_second': 1.632, 'epoch': 2.0}
{'train_runtime': 14236.9553, 'train_samples_per_second': 3.372, 'train_steps_per_second': 0.211, 'train_loss': 0.2816208273569743, 'epoch': 2.0}
CPU times: user 32min 27s, sys: 13min 41s, total: 46min 9s
Wall time: 3h 57min 17s


TrainOutput(global_step=3000, training_loss=0.2816208273569743, metrics={'train_runtime': 14236.9553, 'train_samples_per_second': 3.372, 'train_steps_per_second': 0.211, 'total_flos': 6496406470656000.0, 'train_loss': 0.2816208273569743, 'epoch': 2.0})

In [27]:
finetuned_model.save_pretrained("t5-customer-service")

In [28]:
tokenizer.save_pretrained("t5-customer-service")

('t5-customer-service/tokenizer_config.json',
 't5-customer-service/special_tokens_map.json',
 't5-customer-service/tokenizer.json')

In [29]:
model_path = "t5-customer-service"
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
finetuned_model = finetuned_model.to('cpu')

finetuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [36]:
test_prompt = f'instruction:\n{test}\n\nresponse:\n'
input_text = finetuned_tokenizer(test_prompt,return_tensors="pt").input_ids
output = finetuned_model.generate(
    input_ids=input_text, 
    generation_config=GenerationConfig(max_new_tokens=200, 
    num_beams=1))

model_text_output = finetuned_tokenizer.decode(output[0], skip_special_tokens=True)

print(model_text_output)

I've realized that you're seeking assistance with canceling order Order Number, and I'm here to help you with that. I apologize for any inconvenience this may have caused you. To cancel your order, please follow these steps: 1. Sign into Your Account: Access our Online Company Portal Info and sign in with your order number Order Number. 2. Access Your Order: Once logged in, navigate to the 'Online Order Interaction' or 'Online Order Interaction' section. 3. Identify the Order: Look for the order with the order number Order Number and click on it. 4. Initiate the Cancellation: You should find the option labeled 'Online Order Interaction' associated with the order Order Number. Click on it to proceed with the cancellation. Please select this option. 5. Follow Any Additional Steps:


In [34]:
def get_response(question):
    test_prompt = f'instruction:\n{question}\n\nresponse:\n'
    input_text = finetuned_tokenizer(test_prompt,return_tensors="pt").input_ids
    output = finetuned_model.generate(
        input_ids=input_text, 
        generation_config=GenerationConfig(max_new_tokens=200, 
        num_beams=1))
    return finetuned_tokenizer.decode(output[0], skip_special_tokens=True)

In [35]:
question = "i have a question about cancelling order {{Order Number}}"
get_response(question)

"I've got that you have a question regarding canceling order Order Number. I'm here to assist you with that. Please ask me a question, and I'll provide you with the information you need."

In [37]:
for i in range(0, 5, 1):
    print('Instruction: ' + dataset['test'][i]['instruction'])
    print('Predict. :' + get_response(dataset['test'][i]['instruction']))
    print('Expected: ' + dataset['test'][i]['response'])
    print('=================================\n')

Instruction: how to recover my bloody access key?
Predict. :I'm on the same page, your frustration and urgency in recovering your access key. Let's get you back on track: 1. To initiate the recovery process, please follow these steps: 1. Visit the "Login Page URL" of our platform. 2. Look for the "Forgot Access Key" option and click on it. 3. You will be prompted to enter the email address associated with your account. Follow the instructions provided to reset your access key. Remember to remember to remember to a key reset key, and a key reset key will be a key back. Follow the instructions provided to regain access to your account. If you encounter any difficulties during the process or if you don't receive the email within a reasonable time, please don't hesitate to reach out to me. I'm here to support you every step of the way.
Expected: Absolutely! I understand the importance of recovering your access key and I'm here to assist you every step of the way. Please follow these instru

PEFT

In [70]:
from peft import LoraConfig, get_peft_model, TaskType

In [71]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [72]:
peft_model = get_peft_model(base_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 3,538,944 || all params: 251,116,800 || trainable%: 1.4093
None


In [73]:
import time

output_dir = 'training-peft'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    learning_rate=1e-3,
    num_train_epochs= 3, 
    no_cuda=True,  
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset= tokenized_datasets['test']
)

/Users/johnmoses/miniforge3/envs/mforge39/lib/python3.9/site-packages/transformers/training_args.py:1332: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/Users/johnmoses/miniforge3/envs/mforge39/lib/python3.9/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
from safetensors.torch import load_model, save_model

peft_trainer.train()
save_model(peft_model, "model.safetensors")

In [42]:
import evaluate

In [43]:
rouge = evaluate.load('rouge')

In [47]:
instructions = dataset['test'][0:25]['instruction']
responses = dataset['test'][0:25]['response']

base_model_responses = []
finetuned_model_responses = []
peft_model_responses = []

for instruction in instructions:
    prompt = f""" 
    instruction:
    {instruction}

    response:
    """
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    base_model_outputs = base_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    base_model_output = tokenizer.decode(base_model_outputs[0], skip_special_tokens=True)

    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    finetuned_model_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)


    # peft_model_outputs = peft_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    # peft_model_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    base_model_responses.append(base_model_output)
    finetuned_model_responses.append(finetuned_model_output)
    # peft_model_responses.append(peft_model_output)

In [48]:
zipped_responses = list(zip(responses, base_model_responses, finetuned_model_responses))
df = pd.DataFrame(zipped_responses, columns=['human','base','finetuned'])
df

,human,base,finetuned
0,Absolutely! I understand the importance of rec...,:,"I'm on the same page, your frustration and urg..."
1,Assuredly! I understand that you are looking f...,:,"I'm on the same page, your need to restore the..."
2,Unquestionably! I understand the urgency of re...,I have got to recover my user pwd response:,I'll get right on it! I'm here to assist you i...
3,Sure! I'm here to assist you with resetting yo...,instruction: what do do ot do to reset my user...,I've realized that you're seeking guidance on ...
4,I'm on it! I'm here to guide you through the p...,instruction: how do i retrieve my profile pin ...,"I'm on the same page, your concern about retri..."
5,For sure! I'm here to assist you in retrieving...,:,I'll take care of it! I'm here to assist you i...
6,Oh no! I understand the frustration and urgenc...,instruction: i cannot rettrieve my damn user p...,I'm sorry to hear that you're having trouble r...
7,Unquestionably! I understand your need for inf...,"Ich hoffe, dass ich Informationen über meine A...",I'm on it! I'm here to provide you with all th...
8,Indeed! I'm here to lend you the expertise you...,Instructions: Instruction instruction instruct...,I'm on it! I'm here to assist you in recoverin...
9,Oh no! I understand the frustration of not bei...,instruction: I can't remember my account passw...,I'm sorry to hear that you're having trouble r...


In [49]:
responses = dataset['test'][0:25]['response']

base_model_results = rouge.compute(
    predictions=base_model_responses,
    references=responses,
    use_aggregator=True,
    use_stemmer=True,
)
print('Base\n',base_model_results)

finetuned_model_results = rouge.compute(
    predictions=finetuned_model_responses,
    references=responses,
    use_aggregator=True,
    use_stemmer=True,
)
print('Fine-tuned\n',finetuned_model_results)

# peft_model_results = rouge.compute(
#     predictions=peft_model_responses,
#     references=response,
#     use_aggregator=True,
#     use_stemmer=True,
# )
# print('\PEFT\n',peft_model_results)

Base
 {'rouge1': 0.06125879176286796, 'rouge2': 0.017930397475075, 'rougeL': 0.05169171787061669, 'rougeLsum': 0.05397238667865421}
Fine-tuned
 {'rouge1': 0.5363634333479866, 'rouge2': 0.27311047339567807, 'rougeL': 0.369923933719136, 'rougeLsum': 0.41543613109721367}
